# Hyperliquid Scalping Bot Development Playbook 🚀

## Complete End-to-End Guide for High-Frequency Trading with Rust

This notebook implements the comprehensive scalping playbook for building a production-ready high-frequency trading system using Rust and the Hyperliquid SDK. 

### Architecture Overview

```
┌────────────┐     WS::Book/Trades     ┌────────────┐
│ HL Gateway │ ──────────────────────► │  DataBus   │   (Tokio mpsc)
└────────────┘ ◄────────────────────── └────────────┘
       ▲                                   │
       │      REST::Exchange / Info        ▼
┌────────────┐                      ┌──────────────┐
│ Strategy    │  async state‑loop   │  Risk/MM     │
│ (Scalper)   │  orders  ◄──────────┤  Guards      │
└────────────┘                      └──────┬───────┘
                                           │    snapshots→ S3/Parquet
                                           ▼
                                 ┌──────────────────┐
                                 │ Back‑test engine │
                                 └──────────────────┘
                                           ▼
                                 Grafana / Prometheus
```

### What You'll Build

- **Real-time data streaming** with WebSocket connections
- **Production scalping strategy** with order book analysis
- **High-performance backtesting** using Polars DataFrames
- **Risk management system** with automated safeguards
- **Live trading engine** with monitoring and alerting
- **Complete monitoring stack** with Prometheus/Grafana

---

**⚠️ Important Safety Notice**: This involves real money and live trading. Always test thoroughly in sandbox environments first!

## 1. Environment Setup and Dependencies

### Install Rust Toolchain and Initialize Workspace

First, ensure you have the stable Rust toolchain and create the project workspace.

In [ ]:
// Environment Bootstrap Commands (Run in terminal)
// 
// 1. Install stable toolchain
// rustup default stable
//
// 2. Create workspace (if not exists)
// cargo new --vcs git hl_scalper
//
// 3. Add core dependencies
// cargo add hyperliquid_rust_sdk tokio polars rayon clap dotenvy tracing anyhow
// cargo add ethers rand chrono serde plotters
//
// 4. Verify SDK version
// cargo search hyperliquid_rust_sdk

use std::process::Command;

fn check_environment() -> Result<(), Box<dyn std::error::Error>> {
    println!("🔧 Checking Rust environment...");
    
    // Check Rust version
    let output = Command::new("rustc")
        .arg("--version")
        .output()?;
    
    println!("Rust version: {}", String::from_utf8_lossy(&output.stdout));
    
    // Check Cargo version
    let output = Command::new("cargo")
        .arg("--version")
        .output()?;
        
    println!("Cargo version: {}", String::from_utf8_lossy(&output.stdout));
    
    println!("✅ Environment ready!");
    Ok(())
}

// Run the check
match check_environment() {
    Ok(_) => println!("Environment setup complete!"),
    Err(e) => println!("Error: {}", e),
}

### Cargo.toml Configuration

Your `Cargo.toml` should include these dependencies for the scalping bot:

In [ ]:
// Complete Cargo.toml for Hyperliquid Scalping Bot
/*
[package]
name = "hyperliquid-scalper"
version = "0.1.0"
edition = "2021"

[dependencies]
# Core Hyperliquid SDK
hyperliquid_rust_sdk = "0.6.0"

# Async runtime and utilities
tokio = { version = "1.0", features = ["full"] }
futures = "0.3"

# Data processing and analysis
polars = { version = "0.35", features = ["lazy", "temporal", "parquet"] }
rayon = "1.8"

# Command line interface
clap = { version = "4.0", features = ["derive"] }

# Environment and configuration
dotenvy = "0.15"

# Logging and tracing
tracing = "0.1"
tracing-subscriber = { version = "0.3", features = ["env-filter"] }

# Error handling
anyhow = "1.0"
thiserror = "1.0"

# Blockchain and crypto
ethers = "2.0"

# Utilities
chrono = { version = "0.4", features = ["serde"] }
serde = { version = "1.0", features = ["derive"] }
serde_json = "1.0"
rand = "0.8"

# Visualization and output
plotters = "0.3"
csv = "1.3"

# Monitoring and metrics
prometheus = "0.13"
metrics = "0.21"

# HTTP client
reqwest = { version = "0.11", features = ["json"] }

[dev-dependencies]
quickcheck = "1.0"
quickcheck_macros = "1.0"

[profile.release]
opt-level = 3
lto = true
codegen-units = 1
panic = "abort"
*/

// Let's verify our existing project structure
use std::path::Path;

fn check_project_structure() {
    let paths = [
        "src/strategies/mod.rs",
        "src/strategies/prelude.rs", 
        "src/strategies/live_scalping.rs",
        "src/strategies/scalping.rs",
        "examples/basic_backtest.rs",
        "Cargo.toml"
    ];
    
    println!("📁 Checking project structure:");
    for path in &paths {
        if Path::new(path).exists() {
            println!("✅ {}", path);
        } else {
            println!("❌ {} (missing)", path);
        }
    }
}

check_project_structure();

## 2. SDK Configuration and Authentication

### Environment Variables and Wallet Setup

Configure your Hyperliquid SDK with proper authentication and environment variables.

In [ ]:
// Environment Configuration and SDK Setup
use std::env;
use ethers::signers::LocalWallet;
use anyhow::Result;

// Create .env.example file content
fn generate_env_example() {
    let env_content = r#"
# Hyperliquid Configuration
HL_PK=your_private_key_hex_here
HL_TESTNET=false
HL_BASE_URL=https://api.hyperliquid.xyz

# Risk Management
MAX_POSITION_SIZE=1000.0
MAX_DAILY_LOSS=500.0
DRAWDOWN_LIMIT=0.02

# Strategy Parameters
IMBALANCE_THRESHOLD=0.6
SPREAD_THRESHOLD_BPS=2
EMA_PERIOD_SECONDS=15
KELLY_FRACTION=0.25

# Monitoring
PROMETHEUS_PORT=9090
LOG_LEVEL=info
"#;
    
    println!("📝 .env.example content:");
    println!("{}", env_content);
}

// Wallet configuration helper
fn setup_wallet() -> Result<LocalWallet> {
    dotenvy::dotenv().ok();
    
    let private_key = env::var("HL_PK")
        .map_err(|_| anyhow::anyhow!("HL_PK environment variable not set"))?;
    
    let wallet: LocalWallet = private_key.parse()
        .map_err(|e| anyhow::anyhow!("Invalid private key format: {}", e))?;
    
    println!("✅ Wallet configured successfully");
    println!("Address: {:?}", wallet.address());
    
    Ok(wallet)
}

// Environment validation
fn validate_environment() -> Result<()> {
    let required_vars = [
        "HL_PK",
        "MAX_POSITION_SIZE", 
        "MAX_DAILY_LOSS",
        "IMBALANCE_THRESHOLD"
    ];
    
    println!("🔍 Validating environment variables:");
    
    for var in &required_vars {
        match env::var(var) {
            Ok(value) => {
                // Don't print private key
                if var == &"HL_PK" {
                    println!("✅ {} = [REDACTED]", var);
                } else {
                    println!("✅ {} = {}", var, value);
                }
            }
            Err(_) => {
                println!("❌ {} is missing", var);
                return Err(anyhow::anyhow!("Missing required environment variable: {}", var));
            }
        }
    }
    
    Ok(())
}

// Generate example environment file
generate_env_example();

// Test wallet setup (will fail without proper .env)
println!("\n🔐 Testing wallet configuration:");
match setup_wallet() {
    Ok(_) => println!("Wallet setup successful!"),
    Err(e) => println!("Wallet setup failed: {} (Expected if .env not configured)", e),
}

// Validate environment
println!("\n🔍 Environment validation:");
match validate_environment() {
    Ok(_) => println!("All environment variables configured!"),
    Err(e) => println!("Environment validation failed: {}", e),
}